In [2]:
import pickle

import pandas as pd


from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

from sklearn.pipeline import make_pipeline

import mlflow

In [4]:
# setting the tracking URI to the local file system

mlflow.set_tracking_uri("http://127.0.0.1:5000")

mlflow.set_experiment("green-taxi-experiment")

2025/06/12 23:27:17 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///C:/Users/STUDSENT/Desktop/machine-learning-Ops/04-deployment/batch_deployment/mlruns/1', creation_time=1749763637001, experiment_id='1', last_update_time=1749763637001, lifecycle_stage='active', name='green-taxi-experiment', tags={}>

In [5]:
def read_dataframe(filename: str) -> pd.DataFrame:
    """
    Reads a CSV file and returns a DataFrame.
    Args:
        filename (str): The path to the CSV file.
    """

    data = pd.read_parquet(filename)

    data['duration'] = data.lpep_dropoff_datetime - data.lpep_pickup_datetime

    data.duration = data.duration.dt.total_seconds() / 60
    data = data[(data.duration >= 1) & (data.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    data[categorical] = data[categorical].astype(str)
    return data


In [6]:
def clear_dict(data: pd.DataFrame) -> dict:
    """
    Converts a DataFrame to a dictionary format suitable for DictVectorizer.
    Args:
        data (pd.DataFrame): The input DataFrame.
    Returns:
        dict: A dictionary representation of the DataFrame.
    """
    data['PU_DO'] = data['PULocationID'] + '_' + data['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']

    dicts = data[categorical + numerical].to_dict(orient='records')
    return dicts

In [7]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = clear_dict(df_train)
dict_val = clear_dict(df_val)

In [8]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = root_mean_squared_error(y_pred, y_val)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


2025/06/12 23:28:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run wistful-robin-345 at: http://127.0.0.1:5000/#/experiments/1/runs/8fd3dc15a36d4240bf05b54b75ed1541
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [9]:
from mlflow.tracking import MlflowClient

In [10]:

MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '8fd3dc15a36d4240bf05b54b75ed1541'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)
# client

In [11]:

model_path = client.download_artifacts(run_id=RUN_ID, path='model/')

with open(model_path + '/model.pkl', 'rb') as file:
    dv = pickle.load(file)
    print(dv)

Pipeline(steps=[('dictvectorizer', DictVectorizer()),
                ('randomforestregressor',
                 RandomForestRegressor(max_depth=20, min_samples_leaf=10,
                                       n_jobs=-1, random_state=0))])
